In [148]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
import datetime
import xml.etree.cElementTree as et
from datetime import datetime
from pandas.core.tools.datetimes import to_datetime

from datetime import timedelta
from RNN import RNN
from Transformer import Transformer

from utils import series_to_supervised


In [149]:
def read_xml_data(filename, selected_items):
  tree=et.parse(filename)
  root=tree.getroot()
  #extract selected items
  for child in root:
    if child.tag in selected_items:
      df = pd.DataFrame()
      for elem in child:      
        df1 = pd.DataFrame(elem.attrib, index=[0])
        #df = df.append(df1)
        df = pd.concat([df, df1])
      #First column is the timestamp (dayfirst)      
      #df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], dayfirst=True)
      #write to csv file using the timestamp as index      
      df.to_csv(child.tag+'.csv', index=0)

In [150]:
def read_ts_file(filename):
  #reads csv file where the first column is a timestamp and the index column
  df = pd.read_csv (filename, parse_dates=[0], dayfirst=True, index_col=0)
  return df

In [151]:
def timedf(df):
  #creates a 5 minute interval timeseries dataframe based in index of df
  # df must have a timestamp index
  #time_df: result dataframe with timestamp index
  timestamp = pd.date_range(start=df.index[0], end=df.index[-1]  + timedelta(minutes=4), freq='5T')
  time_df = pd.DataFrame({'timestamp':timestamp})
  time_df.set_index('timestamp', inplace=True)
  return time_df

In [152]:
def find_gaps(df, greaterthan=5, units='m'):
  # find gaps relative to index, index must be a datetime field
  # greaterthan is the number of time units to be considered a gap
  # units 'm'=minutes, 'h'=hours
  i = 0
  gaps_df = pd.DataFrame()
  while i < len(df) - 1:
    ts = df.index[i]
    next_ts = df.index[i+1]
    duration = next_ts - ts
    if duration > np.timedelta64(greaterthan, units): 
      begin_gap = ts
      end_gap = next_ts
      gaps_df = gaps_df.append({'From': begin_gap, 'To': end_gap, 'Duration': duration}, ignore_index=True)
    i = i + 1
  gaps_df.sort_values(by=['Duration'], ascending=False, inplace=True)
  return gaps_df

In [153]:
def impute_mean(df, column, by='hour'):
  # impute with mean by hours
  # in the future by could be another mean grouping criterion
  df[by] = df.index.hour
  df[column] = df.groupby(by)[column].apply(lambda x: x.fillna(x.mean()))
  df.drop(by, axis=1, inplace=True)

In [154]:
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars=1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [155]:
#selected_items = ['glucose_level','bolus','meal']
read_xml_data(filename='c://aadm/563-ws-training.xml', selected_items=['glucose_level','bolus','meal'])

In [156]:
glucose_train = read_ts_file('glucose_level.csv')
glucose_train.rename(columns={"ts": "timestamp", "value": "glucose"}, inplace=True)
glucose_train.describe()

,glucose
count,12124.00000
mean,146.07712
std,49.68904
min,40.00000
25%,108.00000
50%,140.00000
75%,177.00000
max,400.00000


In [157]:
#Finding the length of the complete time series
time_df = timedf(glucose_train)
print('Missing intervals: ', len(time_df) - len(glucose_train) )

Missing intervals:  974


In [158]:
#Adding NA in the whole range of cgm-training
glucose_miss=glucose_train.resample('5T').mean()
glucose_miss.head()

,glucose
ts,
2021-09-13 12:30:00,219.0
2021-09-13 12:35:00,229.0
2021-09-13 12:40:00,224.0
2021-09-13 12:45:00,221.0
2021-09-13 12:50:00,215.0


In [159]:
# #mean imputation
#impute_mean(glucose_miss, 'glucose')
#glucose_train=glucose_miss
glucose_train= glucose_miss.interpolate(method="spline",order=3)
glucose_train.describe()

,glucose
count,13098.000000
mean,151.670453
std,65.752426
min,24.650677
25%,109.000000
50%,141.000000
75%,180.000000
max,570.081294


In [160]:
#Dickey-Fuller Test
from statsmodels.tsa.stattools import adfuller
X = glucose_train.glucose.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -10.198214
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [161]:
# Function to print out results in customised manner
from statsmodels.tsa.stattools import kpss
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c', nlags="auto")
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','#Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)
kpss_test(X)

Results of KPSS Test:
Test Statistic            0.466732
p-value                   0.049159
#Lags Used               69.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


In [162]:
#glucose_train= glucose_train.interpolate(method="spline",order=3)
#glucose_train.describe()
glucose_train.head()
glucose_train['glucose']=glucose_train['glucose'].diff().diff()

In [163]:
# # spline smoothing
# import rpy2
# import rpy2.rinterface
# %load_ext rpy2.ipython
# import rpy2.robjects as robjects
# from rpy2.robjects.packages import importr
# splines = importr('splines') 
# x_train=np.arange(len(glucose_train))
# y_train=glucose_train['glucose']
# r_y = robjects.FloatVector(y_train)
# r_x = robjects.FloatVector(x_train)
# r_smooth_spline = robjects.r['smooth.spline'] #extract R function# run smoothing function
# spline1 = r_smooth_spline(x=r_x,y=r_y, spar=.01)
# ySpline=np.array(robjects.r['predict'](spline1,robjects.FloatVector(x_train)).rx2('y'))
# # print(ySpline)
# # plt.figure(figsize=(12,6))
# # plt.scatter(x_train,y_train,c="blue")
# # plt.plot(x_train,ySpline,c='red')

In [164]:
# #Imputation using Kalman smoothing and spline smoothing
# from tsmoothie.smoother import *
# from tsmoothie.utils_func import create_windows
# #smoother1 = SplineSmoother(n_knots=100, spline_type='cubic_spline')
# #smoother1.smooth(cgmtrainclean[['glucose']].T)
# smoother = KalmanSmoother(component='level_season', 
#                           component_noise={'level':0.1, 'season':0.1},n_seasons=7)
# smoother.smooth(glucose_miss[['glucose']].T)
# glucosekf=smoother.smooth_data[0]
# glucose_train=glucose_miss
# glucose_train['glucose']=glucosekf
# glucose_train['glucose']=ySpline
# #smoother1.smooth_data[0].mean()

In [165]:
#selected_items = ['glucose_level',"bolus','meal']
read_xml_data(filename='c://aadm/591-ws-testing.xml', selected_items=['glucose_level','bolus','meal'])

In [17]:
glucose_test = read_ts_file('glucose_level.csv')
glucose_test.rename(columns={"ts": "timestamp", "value": "glucose"}, inplace=True)
max=glucose_test.max()
min=glucose_test.min()

In [18]:
#Finding the length of the complete time series
time_df = timedf(glucose_test)
print('Missing intervals: ', len(time_df) - len(glucose_test) )

Missing intervals:  87


In [19]:
#Adding NA in the whole range of cgm-testing
glucose_miss=glucose_test.resample('5T').mean()
glucose_miss.head()

,glucose
ts,
2022-01-14 00:00:00,283.0
2022-01-14 00:05:00,282.0
2022-01-14 00:10:00,281.0
2022-01-14 00:15:00,277.0
2022-01-14 00:20:00,267.0


In [20]:
# #mean imputation
impute_mean(glucose_miss, 'glucose')
glucose_test=glucose_miss
glucose_test.head()

,glucose
ts,
2022-01-14 00:00:00,283.0
2022-01-14 00:05:00,282.0
2022-01-14 00:10:00,281.0
2022-01-14 00:15:00,277.0
2022-01-14 00:20:00,267.0


In [21]:
#glucose_test= glucose_test.interpolate(method="spline",order=3)
#glucose_test.describe()

In [22]:
# spline smoothing
import rpy2
import rpy2.rinterface
%load_ext rpy2.ipython
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
splines = importr('splines') 
x_train=np.arange(len(glucose_test))
y_train=glucose_test['glucose']
r_y = robjects.FloatVector(y_train)
r_x = robjects.FloatVector(x_train)
r_smooth_spline = robjects.r['smooth.spline'] #extract R function# run smoothing function
spline1 = r_smooth_spline(x=r_x,y=r_y, spar=.01)
ySpline=np.array(robjects.r['predict'](spline1,robjects.FloatVector(x_train)).rx2('y'))
# print(ySpline)
# plt.figure(figsize=(12,6))
# plt.scatter(x_train,y_train,c="blue")
# plt.plot(x_train,ySpline,c='red')

Unable to determine R home: [WinError 2] The system cannot find the file specified
C:\Users\eacun\anaconda3\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [23]:
# #Imputation using Kalman smoothing and spline smoothing
# from tsmoothie.smoother import *
# from tsmoothie.utils_func import create_windows
# #smoother1 = SplineSmoother(n_knots=100, spline_type='cubic_spline')
# #smoother1.smooth(cgmtrainclean[['glucose']].T)
# smoother = KalmanSmoother(component='level_season', 
#                           component_noise={'level':0.1, 'season':0.1},n_seasons=7)
# smoother.smooth(glucose_miss[['glucose']].T)
# glucosekf=smoother.smooth_data[0]
# glucose_test=glucose_miss
# glucose_test['glucose']=glucosekf
glucose_test['glucose']=ySpline
# #smoother1.smooth_data[0].mean()

In [24]:
#Normalizing the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
datatrain = np.array(glucose_train.values.astype('float32'))
datatest = np.array(glucose_test.values.astype('float32'))
scaler = MinMaxScaler(feature_range=(0, 1))
datatrain = scaler.fit_transform(datatrain).flatten()
datatest = scaler.fit_transform(datatest).flatten()
#n = len(data)
train_data=pd.DataFrame(datatrain)
test_data=pd.DataFrame(datatest)
print(max,min)

glucose    291
dtype: int64 glucose    43
dtype: int64


In [25]:
data1=series_to_supervised(train_data, n_in=12, n_out=6, dropnan=True)
data2=series_to_supervised(test_data, n_in=12, n_out=6, dropnan=True)
train=data1.values
X_train,y_train=train[:, 0:12],train[:, 12:]
test=data2.values
X_test,y_test=test[:, 0:12],test[:, 12:]
ytest=y_test
print(y_train.shape)
print("test shape:",X_test.shape)
# reshape input to be 3D [samples, timesteps, features]
X_train_reshaped = X_train.reshape((-1,12,1))
X_test_reshaped = X_test.reshape((-1,12,1))
print(X_train_reshaped.shape,X_test_reshaped.shape)
y_train_reshaped = y_train
print(y_train_reshaped.shape)
y_test_reshaped = y_test

(12738, 6)
test shape: (2830, 12)
(12738, 12, 1) (2830, 12, 1)
(12738, 6)


In [26]:
# Testing the RNN-LSTM
#rnn = RNN()
#rnn.train(X_train_reshaped,y_train_reshaped)
#_, rmse_result, mae_result, smape_result, r2_result = rnn.evaluate(X_test_reshaped,y_test_reshaped,max,min)
#print('Result \n RMSE = %.2f  \n MAE = %.2f \n R2 = %.1f [%%]' % (rmse_result,
#                                                                            mae_result,                                                                          
#                                                                            r2_result*100))

In [27]:
## Testing the Transformer
import time
start_time = time.time()
look_back = 12
tr = Transformer()
tr.train(X_train_reshaped,y_train_reshaped)
a=tr.evaluate(X_test_reshaped,y_test_reshaped)

print(a)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12, 2)]      0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 12, 2)       4           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 12, 2)       11266       ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 12, 2)       4           ['tf.__operators__.add_6[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv1d_6 (Conv1D)              (None, 12, 4)        12          ['layer_normalization_7[0][0]']  
                                                                                                  
 dropout_7 (Dropout)            (None, 12, 4)        0           ['conv1d_6[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 12, 2)        10          ['dropout_7[0][0]']              
          

ValueError: in user code:

    File "C:\Users\eacun\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\eacun\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\eacun\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\eacun\anaconda3\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\eacun\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "layer_normalization" (type LayerNormalization).
    
    Dimension 2 in both shapes must be equal, but are 2 and 1.

In [ ]:
max=np.array(max)
min=np.array(min)

In [ ]:
#Denormalizing prediction
forec1=a*(max-min)+min
print(forec1)

In [ ]:
#Denomalizing the actual values
actual=ytest*(max-min)+min
print(actual)

In [ ]:
#Computing the RMSE
forec1=np.array(forec1)
actual=np.array(actual)
diff=actual-forec1
#print(diff.shape)
#np.sqrt(np.mean((diff)**2,axis=0))
print("RMSE:",np.sqrt(np.mean((diff)**2,axis=0)).mean())

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))